### ✳️ Required User Inputs

In [1]:
# Source
source_filename = "3commas-telegram-2021-03-09.json"

# Removes messages with the following substrings
remove = ["Start command received", "Stop command received", 
          "Bot is turned on", "Bot is turned off"]

### Setting up the stage!

In [2]:
import json
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
# Grabbing the data
folder = "commas3/data/"
source = folder + source_filename
data = pd.read_json(source)

# Display full column width
pd.set_option('display.max_colwidth', None)

In [4]:
def SliceData(df, timeframe):
    today = dt.datetime.today().date()
    start_date = today - dt.timedelta(timeframe)
    df_period = df.loc[start_date:today]
    return df_period

### Clean Up

In [5]:
data = data["messages"]
data = pd.json_normalize(data)
data["date"] = pd.to_datetime(data['date'])
data.set_index("date", inplace=True)
data = data[data["from_id"] == 4882035576]
data.drop(columns=["id", "type", "from", "from_id"], inplace=True)
data.rename(columns={"text": "message"}, inplace=True)
data = data[7:]
data["message"] = data["message"].apply(lambda row: \
                                     "".join([x if isinstance(x, str) else x["text"] for x in row]))                                      
data = data[~data['message'].str.contains("|".join(remove))]


### Add Features

In [6]:
data["bot"] = data['message'].str.split(":")
data["bot"] = data["bot"].str[0]
data["bot"] = data["bot"].apply(lambda x: x.split("(")[0] if "(" in x else x)

mask = data["message"].str.contains("Error")
data.loc[mask, "status"] = "error"

mask = data["status"] == "error"
data.loc[mask, "alert"] = data.loc[mask, "message"].apply(lambda x: x.split("Error")[1])
data.loc[mask, "alert"] = data.loc[mask, "alert"].apply(lambda x: x.split(":")[1])
data.loc[mask, "alert"] = data.loc[mask, "alert"].apply(lambda x: "IP banned" if "banned" in x \
                                                    else(x.split(".")[0] if "." in x \
                                                         else (x.split("#")[0] if "#" in x \
                                                               else x)))
data["alert"] = data["alert"].str.strip()

mask = data["message"].str.contains("Conditions not met")
data.loc[mask, "status"] = "settings"
data.loc[mask, "alert"] = "conditions"

mask = data["message"].str.contains("Maximum active deals reached")
data.loc[mask, "status"] = "settings"
data.loc[mask, "alert"] = "max deals"

### Save clean data to CSV File

In [7]:
destination_filename = source_filename.split(".")[0] + "_clean.csv"
destination = folder + destination_filename
data.to_csv(destination, index = True) 

# Data Explorer

In [8]:
df = SliceData(data, 7)
df.groupby([pd.Grouper(freq='D'), "alert"])["alert"].count().sort_index(ascending=False)

date        alert                   
2021-03-08  max deals                    63
            conditions                   59
            Insufficient funds           28
2021-03-07  max deals                    70
            conditions                   37
2021-03-06  max deals                    68
            conditions                   27
            Market is closed/offline      1
2021-03-05  max deals                    23
            conditions                   53
            Insufficient funds          204
2021-03-04  max deals                    84
            conditions                   37
            Insufficient funds          140
2021-03-03  max deals                    98
            conditions                   52
            Insufficient funds            1
2021-03-02  max deals                    24
            conditions                   44
Name: alert, dtype: int64

In [9]:
df = SliceData(data, 5)
df = df[df["alert"] != "conditions"]
df.groupby([pd.Grouper(freq='D'), "bot", "alert"])["alert"].count().sort_index(ascending=False)

date        bot                                         alert                   
2021-03-08  TBT USDT P-B D-21-01-06-16-02               max deals                    1
            TBT BTC P-Q D-21-01-10-18-00                Insufficient funds           4
            QFL-PT BTC P-Q D-21-02-28-14-57             Insufficient funds           8
            QFL-DT BTC P-Q D-21-02-08-20-38             Insufficient funds          16
            HirnVIP BTC P-Q D-21-01-29-17-36            max deals                   15
            Blockparty USDT P-Q TP25 D-21-01-30-20-12   max deals                   47
2021-03-07  HirnVIP BTC P-Q D-21-01-29-17-36            max deals                   18
            Blockparty USDT P-Q TP25 D-21-01-30-20-12   max deals                   52
2021-03-06  HirnVIP BTC P-Q D-21-01-29-17-36            max deals                   15
            Blockparty USDT P-Q TP25 D-21-01-30-20-12   Market is closed/offline     1
            Blockparty USDT P-Q TP25 D-21-01-30-2

In [10]:
# mask = data["message"].str.contains("|".join(settings_alerts))